# Data Exploration: Online Retail II

This notebook explores the Online Retail II dataset to understand its structure, identify data quality issues, and inform data cleaning decisions prior to KPI calculation and modeling.

The goal of this analysis is not to build metrics or models yet, but to document observable inconsistencies and justify downstream cleaning rules.

In [36]:
import pandas as pd

## Dataset Overview

The Online Retail II dataset contains transactional data from an online retail business between December 2009 and December 2011.

Each row represents a product-level line item within a customer invoice. A single invoice may therefore span multiple rows if multiple products were purchased in one order.

In [ ]:
df = pd.read_excel("/content/online_retail_II.xlsx")

## Initial Data Structure

In this section, we inspect the basic structure of the dataset, including column names, data types, and the presence of missing values.

This helps establish a baseline understanding before deeper inspection of individual fields.

In [37]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525461 entries, 0 to 525460
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      525461 non-null  object        
 1   StockCode    525461 non-null  object        
 2   Description  522533 non-null  object        
 3   Quantity     525461 non-null  int64         
 4   InvoiceDate  525461 non-null  datetime64[ns]
 5   Price        525461 non-null  float64       
 6   Customer ID  417534 non-null  float64       
 7   Country      525461 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 32.1+ MB


## Invoice Number Inspection

Invoice numbers are examined to identify patterns that may indicate cancelled or reversed transactions.

In particular, invoices beginning with the letter "C" are commonly used in retail datasets to denote cancellations.

In [ ]:
# looking at a few Invoice No.
df['Invoice'].head(10)

In [ ]:
# Check datatype of Invoice
df['Invoice'].dtype

In [39]:
# Find invoices starting with 'C' (cancellations)
cancelled = df[df['Invoice'].astype(str).str.startswith('C', na=False)]
cancelled.shape

(10206, 8)

In [40]:
cancelled[['Invoice', 'Quantity', 'Price']].head()

,Invoice,Quantity,Price
178,C489449,-12,2.95
179,C489449,-6,1.65
180,C489449,-4,4.25
181,C489449,-6,2.10
182,C489449,-12,2.95


**Observation**

Invoices prefixed with "C" are present and are frequently associated with negative quantities. These rows likely represent cancelled transactions rather than completed sales.

## Quantity Inspection

The `Quantity` field is inspected to identify invalid or non-standard values, such as negative or zero quantities, which may indicate returns, cancellations, or data entry issues.

In [41]:
# Basic stats
df['Quantity'].describe()

,Quantity
count,525461.000000
mean,10.337667
std,107.424110
min,-9600.000000
25%,1.000000
50%,3.000000
75%,10.000000
max,19152.000000


In [42]:
# Count problematic rows
(df['Quantity'] <= 0).sum()

np.int64(12326)

In [43]:
# Peek at some negative quantities
df[df['Quantity'] <= 0].head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
178,C489449,22087,PAPER BUNTING WHITE LACE,-12,2009-12-01 10:33:00,2.95,16321.0,Australia
179,C489449,85206A,CREAM FELT EASTER EGG BASKET,-6,2009-12-01 10:33:00,1.65,16321.0,Australia
180,C489449,21895,POTTING SHED SOW 'N' GROW SET,-4,2009-12-01 10:33:00,4.25,16321.0,Australia
181,C489449,21896,POTTING SHED TWINE,-6,2009-12-01 10:33:00,2.10,16321.0,Australia
182,C489449,22083,PAPER CHAIN KIT RETRO SPOT,-12,2009-12-01 10:33:00,2.95,16321.0,Australia


In [ ]:
df['Quantity'].quantile([0.01, 0.99])

**Observation**

Negative and zero quantities are present in the dataset. These values do not represent valid sales activity and would distort volume and revenue metrics if retained.

## Price Inspection

The `Price` field is examined to identify zero or negative values that would invalidate revenue calculations.

In [44]:
df['Price'].describe()

,Price
count,525461.000000
mean,4.688834
std,146.126914
min,-53594.360000
25%,1.250000
50%,2.100000
75%,4.210000
max,25111.090000


In [45]:
(df['Price'] <= 0).sum()

np.int64(3690)

In [46]:
df[df['Price'] <= 0].head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
263,489464,21733,85123a mixed,-96,2009-12-01 10:52:00,0.0,NaN,United Kingdom
283,489463,71477,short,-240,2009-12-01 10:52:00,0.0,NaN,United Kingdom
284,489467,85123A,21733 mixed,-192,2009-12-01 10:53:00,0.0,NaN,United Kingdom
470,489521,21646,NaN,-50,2009-12-01 11:44:00,0.0,NaN,United Kingdom
3114,489655,20683,NaN,-44,2009-12-01 17:26:00,0.0,NaN,United Kingdom


In [ ]:
df['Price'].quantile([0.01, 0.99])

**Observation**

Rows with zero or negative unit prices are observed. These rows do not represent revenue-generating transactions and should be excluded from KPI calculations.

## Product Description Inspection

The `Description` field is reviewed to understand the prevalence of missing or empty product descriptions and whether this affects downstream analysis.


In [47]:
# Missing descriptions
df['Description'].isna().sum()

np.int64(2928)

In [48]:
# Empty strings (sometimes different from NaN)
(df['Description'].astype(str).str.strip() == '').sum()

np.int64(0)

In [49]:
# Look at some missing ones
df[df['Description'].isna()].head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
470,489521,21646,NaN,-50,2009-12-01 11:44:00,0.0,NaN,United Kingdom
3114,489655,20683,NaN,-44,2009-12-01 17:26:00,0.0,NaN,United Kingdom
3161,489659,21350,NaN,230,2009-12-01 17:39:00,0.0,NaN,United Kingdom
3731,489781,84292,NaN,17,2009-12-02 11:45:00,0.0,NaN,United Kingdom
4296,489806,18010,NaN,-770,2009-12-02 12:42:00,0.0,NaN,United Kingdom


**Observation**

Missing descriptions are present but frequently coincide with invalid price or quantity values. Since product descriptions are not required for KPI computation, no additional imputation is planned.

## Customer Identifier Inspection

Customer identifiers are critical for customer-level KPIs, segmentation, and retention analysis. This section evaluates the extent of missing customer IDs.


In [50]:
df['Customer ID'].isna().sum()

np.int64(107927)

In [51]:
# Percentage missing
df['Customer ID'].isna().mean() * 100


np.float64(20.53948818275762)

In [52]:
# Inspect rows with missing CustomerID
df[df['Customer ID'].isna()].head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
263,489464,21733,85123a mixed,-96,2009-12-01 10:52:00,0.00,NaN,United Kingdom
283,489463,71477,short,-240,2009-12-01 10:52:00,0.00,NaN,United Kingdom
284,489467,85123A,21733 mixed,-192,2009-12-01 10:53:00,0.00,NaN,United Kingdom
470,489521,21646,NaN,-50,2009-12-01 11:44:00,0.00,NaN,United Kingdom
577,489525,85226C,BLUE PULL BACK RACING CAR,1,2009-12-01 11:49:00,0.55,NaN,United Kingdom


**Observation**

A significant portion of rows contain missing `Customer ID` values. Retaining these rows would prevent accurate customer-level analysis, motivating their removal during cleaning.


## Invoice Date Inspection

The `InvoiceDate` field is examined to confirm correct parsing and to understand the overall time span of the dataset.

In [ ]:
# Check type
df['InvoiceDate'].dtype

In [53]:
# Convert to datetime safely (do not overwrite yet)
invoice_dates = pd.to_datetime(df['InvoiceDate'], errors='coerce')

In [54]:
# How many failed to parse?
invoice_dates.isna().sum()

np.int64(0)

In [55]:
# Date range
invoice_dates.min(), invoice_dates.max()

(Timestamp('2009-12-01 07:45:00'), Timestamp('2010-12-09 20:01:00'))

**Observation**

All invoice dates successfully parse as valid timestamps. The dataset spans multiple years, enabling time-series aggregation and trend analysis.


## Country Field Inspection

The `Country` field is reviewed for inconsistencies, unexpected values, or data entry errors.


In [56]:
# Number of unique countries
df['Country'].nunique()

40

In [57]:
# Top countries by frequency
df['Country'].value_counts().head(10)

,count
Country,
United Kingdom,485852
EIRE,9670
Germany,8129
France,5772
Netherlands,2769
Spain,1278
Switzerland,1187
Portugal,1101
Belgium,1054


In [ ]:
# Bottom (rare ones)
df['Country'].value_counts().tail(10)

**Observation**

Country values appear consistent and well-formed. Rare countries are retained to preserve full geographic coverage.


## Summary of Data Quality Findings

Based on exploratory analysis, the following data quality issues were identified:

- Cancelled invoices indicated by invoice numbers starting with "C"
- Negative or zero quantities representing returns or invalid transactions
- Zero or negative unit prices that invalidate revenue calculations
- Missing customer identifiers that prevent customer-level analysis

These observations directly inform the data cleaning rules documented separately in `cleaning_rules.md`.